In [1]:
from IPython.core.display import HTML, Image, Markdown
import sys
import json
sys.path.append("microTC")
from microtc.textmodel import TextModel
from itertools import combinations


from graphviz import Digraph
dot = Digraph(comment='microtc pipeline', format="png")
# dot.engine = 'circo'
dot.graph_attr['rankdir'] = 'LR'
dot.node('i', '', style="invis")
dot.node('n', 'Normalización')
dot.node('t', 'Tokenización')
dot.node('w', 'Pesado')
dot.node('c', 'Clasificación')
dot.node('o', '', style="invis")

dot.edge('i', 'n', label="texto entrada")
dot.edge('n', 't', label="texto normalizado")
dot.edge('t', 'w', label="bolsa de palabras")
dot.edge('w', 'c', label="vector con pesos")
dot.edge('c', 'o', label="clase")

pipeline = dot.render("fig-pipeline", view=False)


# Clasificación de texto #
## Un enfoque basado en $\mu TC$ ##


**Seminario de la Sociedad Matemática Mexicana SMM 2016**


<div>
    Eric Sadit Téllez Avila INFOTEC 
    <estellezav@conacyt.mx> <br/>
       CONACyT -- INFOTEC
</div>

# Agenda #
- ¿Qué es $\mu TC$
- ¿En qué consiste la tarea de clasificación de texto?
- ¿Cómo esta compuesto $\mu TC$?
- Estado del arte
- Cómo se compara $\mu TC$ con el estado del arte
- Qué falta en $\mu TC$
- Ejemplos


## Categorización de texto ##
El problema consiste en, dado un texto $d$, determinar la(s) categoría(s) a la que pertenece en un conjunto $C$ de categorias, previamente conocido.

Más formalmente:

Dado un conjunto de categorias $\cal{C} = \{c_1, ..., c_m\}$, determinar el subconjunto de categorias
$C_d \in \wp(\cal{C})$ a las que pertenece $d$. 

Notese que $C_t$ puede ser vacio o $\cal{C}$.



## Clasificación de texto ##
La _clasificación_ de texto es una especialización del problema de categorización, donde $|C_d| = 1$, esto es $d$ solo puede ser asignado a una categoría.

Es un problema de interés en la industria y la acádemia, con aplicaciones variadas a distintas áreas del conocimiento.

- Análisis de sentimiento
- Determinación de autoría, e.g., género, edad, estilo, etc.
- Detección de spam
- Categorización de noticias
- Clasificación de idioma

# Procesamiento de Lenguaje Natural #


Un documento $d=s_1\cdots s_n$ es simplemente la concatenación de símbolos $s \in \Sigma$. Donde, $\Sigma$ es un _alfabeto_ de tamaño $\sigma = |\Sigma|$

Notese qué el número de textos posibles de tamaño $n$ es $\sigma^n$, por ejemplo, limitados a texto en inglés en Twitter se tienen
    $$ 26^{140} \simeq 1.248 \times 10^{198} $$

Sin emabargo, en lenguaje natural, este número no suele ser tan grande:
  - existen reglas sobre que símbolos se pueden unir
  - más aún, hay noción de _terminos_ o _palabras_, i.e., _morfología_
  - también, hay reglas sobre como las palabras se pueden combinar, i.e., _sintaxis y gramática_

Sin embargo, es un problema sumamente complicado, hay muchas reglas, variantes, excepciones, errores, etc.

Y por si fuera poco, aunque los conceptos existen en esencia, aparecen de manera diferente en todos los lenguajes


Además, esta el problema semántico:

- un término $s_i$ tiene significados diferentes (antónimos)
- lo contrario también existe, $s_i \not= s_j$ pero que son idénticos en significado (sinónimos)
- en ambos casos, el significado preciso depende del contexto
- también hay casos _aproximados_ de todo lo anterior
- ironias, sarcamos, etc.

... hay muchísimos problemas abiertos. **NLP** es complicado, de hecho es _AI-complete_


# Nuestro Enfoque #
Por su complejidad, trabajar en NLP tiene una gran cantidad de problemas abiertos, en particular nosotros nos enfocamos en la clasificación de texto escrito de manera informal (e.g., Twitter).

Para esto se utiliza un _pipeline_ estándar

![Pipeline](fig-pipeline.png)

No es la única opción, pero fijar el pipeline es útil como ya se expondrá.


El enfoque teórico suele ser muy complicado, y en realidad poco efectivo en la práctica, dadas las simplificaciones necesarias para hacerlo manejable

- Lógica
- Lingüistica
- Semántica

El enfoque práctico supone muchas cosas, en particular es un tanto casuístico:
    
- Se fija el lenguaje
- Se fija el problema
- Y la raíz de todos los males, muchas veces se supone que entre más técnicas sofísticadas se usen, mejores resultados se tendrán

En ambos enfoques se suele suponer que ausencia de errores de diferentes fuentes, sin embargo, es la regla cuando el texto que se analiza fue generado por usuarios de una red social, e.g. Twitter.

# ¿Qué es $\mu TC$? #
micro TC o $\mu TC$ es un clasificador de texto desarrollado en
el _Laboratorio de Análisis Computacional de Grandes Cúmulos de Información_
(o _Laboratorio de BigDATA_) de INFOTEC, sede Aguascalientes.

Esta disponible para ser clonado en [https://github.com/INGEOTEC/microTC](https://github.com/INGEOTEC/microTC). Esta escrito en Python 3.5 para sacar ventaja de unicode. También se puede instalar utilizando `pip` y `conda`.


En particular, nuestro enfoque se basa en _aprendizaje computacional_ y _optimización combinatoria_. Hemos probado que este esquema es muy competitivo en la práctica. Además, con la adecuada selección de las funciones podemos lograr que 
$\mu TC$ se independiente del lenguaje y robusto a errores.

Esta compuesto por:
- una serie de funciones de transformación de texto
- una serie de tokenizadores
- filtros de palabras y
- algoritmos de pesado de términos

Todo esto orquestado mediante un algoritmo de optimización combinatoria


## Lista de parametros ##

### Normalizadores multilenguaje ###

|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|	del-punc | yes, no | Determina si las puntuaciones deben removerse |
|	del-d1   | yes, no | Determina si se deben borrar letras repetidas |
|	del-diac | yes, no | Determina si los simbolos que no ocupan espacios deben ser removidos |
|	lc       | yes, no | Determina si los símbolos deben ser normalizados en minúsculas |
|	emo      | remove, group, none | Controla como deben tratarse los emoticones |
|	num      | remove, group, none | `........................` números |
|	url      | remove, group, none | `........................` urls |
|	usr      | remove, group, none | `........................` usuarios |

configuraciones: 1296



### Normalizadores dependientes del lenguaje ###

|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|	stem   | yes, no | Determina si a las palabras se les aplica _stemming_. |
|	neg    | yes, no | Determina si los operadores de negación son manejados de manera especial |
|	sw | remove, group, none | Controla como los _stopwords_ son manejados |

configuraciones: 12


### Tokenizadores ###
Los tokenizadores son en realidad una lista de tokenizadores, y están definidos tokenizer un elemento en $\wp{(\text{n-words} \cup \text{q-grams} \cup \text{skip-grams})} \setminus \{\emptyset\}$

|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|	n-words    | $\{1,2,3\}$      | Longitud de n-gramas de palabras (n-words) |
|	q-grams  | $\{1,2,3,4,5,6,7\}$ | Longitud de q-gramas de caracteres) |
|	skip-grams  | $\{(2,1), (3, 1), (2, 2), (3, 2)\}$ | Lista de skip-grams|

configuraciones: 16383

### Parametros para pesado ###
|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|token_min_filter | $\{0.01, 0.03, 0.1, 0.30, -1, -5, -10\}$ | Filtro de frequencias bajas |
|token_max_filter | $\{0.9, 99, 1.0\}$ | Filtro de frequencias altas |
|	tfidf    | yes, no | Determina si se debe realizar un pesado TFIDF de terminos |

configuraciones = 42

In [6]:
conf = 42 * 16383 * 12 * 1292
time = conf * 10 / 60 / 24 / 365.25
print(conf, time)

10668085344 202830.73511293635


Con esto tenemos un total de $10,668,085,344$ configuraciones. Dependiendo del tamaño de cada colección, cada configuración se evalua en tiempo diferente.

Una tarea típica de análisis de sentimientos tiene un costo por configuración de cerca de 10 min. en una computadora de relativamente nueva, i.e., Intel(R) Xeon(R) CPU E5-2640 v3 @ 2.60GHz.

Esto da un total de $202,830.74$ años de tiempo de cómputo.

Un enfoque _naïve_ requiere una cantidad enorme de computadoras para parallelizar y distribuir este proceso, por esta razón, es mejor utilizar algoritmos eficientes para optimizar la búsqueda de la mejor configuración


In [87]:
from sympy import symbols, sin, cos, exp, tan, sqrt
from sympy.plotting import plot, plot3d
from matplotlib import pyplot
x, y = symbols('x y')
plot(-x**2+200, axis=False, show=False).save("opt-1a.png")
plot(sin(x)*sin(1/(x*x)), axis=False, show=False).save('opt-1b.png')
plot3d((-x**2 - y**2, (x, -5, 5), (y, -5, 5)), axis=False, show=False).save('opt-2a.png')
plot3d(sin(x*y), axis=False, show=False).save('opt-2b.png')

# Optimización combinatoria #
Para tener algo práctico utilizamos una aproximación a encontrar la configuración óptima, en a lo más unas cuantas horas. Para esto necesitamos un espacio de búsqueda (dominio) y una función de aptitud.

El espacio de búsqueda son todas las configuraciones.

La función de aptitud es una médida de _que tan atinado_ es nuestro clasificador. Deseamos seleccionar la configuración que maximice la función de aptitud.


## Maximizando una función ##
|fácil|un poco más complicado|
|-|-|
|<img src="opt-1a.png" style="width:90%" />|<img src="opt-1b.png" style="width:90%" />|
|<img src="opt-2a.png" style="width:90%" />|<img src="opt-2b.png" style="width:90%" />|

La realidad es que tenemos algo de mucha más alta dimensión, y que existe en un espacio sin coordenadas (solo hay parametros)

Afortunadamente, se puede samplear el espacio, se pueden definir vecindades de configuraciones.

# Resultados Experimentales #
![datasets](descripcion-datasets.png)

![performance](performance-competencias.png)

![performance 2](performance2.png)

![performance 3](performance3.png)

# Gracias por su atención #

## ¿Comentarios y preguntas? ##


- Para más información sobre el _Posgrado en Ciencias en Ciencia de Datos_ aquí en Aguascalientes: <eric.tellez@infotec.mx>
- Esta presentación la puedes obtener en: <https://github.com/INGEOTEC/pres-MSA-SMM2016>
- $\mu TC$ lo puedes conseguir en: <https://github.com/INGEOTEC/microTC>
- Página de INFOTEC <https://www.infotec.mx/>

# Slides de soporte #

## Sobre el pesado ##
El pesado de tokens esta fijo a TFIDF. Su nombre viene de la formulación $tf \times idf$

$tf$ es _term frequency_; es una medida de importancia **local** del término $t$ en el documento $d$, de manera normalizada esta definida como:
    $$tf(t,d) = \frac{freq(t, d)}{\max_{w \in d}{freq(w, d)}}$$
entre más veces aparece en el documento $d$, $t$ es más importante

$idf$ quiere decir _inverse document frequency_; es una medida **global** a la colección $D$, esta definida como:
$$ idf(t,d) = log{\frac{|D|}{1+|{d \in D: t \in d}|}} $$
entre más veces aparece $t$ en la colección, el término es más común y menos discriminante; por lo tanto, menos importante

## Sobre el clasificador ##
El clasificador es un algoritmo de aprendizaje computacional que dado un objeto, decide finalmente la etiqueta o clase de ese objeto. Tiene dos etapas bien definidas

- **Entrenamiento.** Dado un conjunto de ejemplos en un espacio vectorial, con etiquetas, el algoritmo intenta _aprender_ las características que definen cada clase
- **Predicción.** La idea es que una vez entrenado, el algoritmo puede recibir objetos no vistos durante la etapa de entrenamiento y asignales la clase adecuada

En particular, esta fijo como un _Support Vector Machine_ (SVM) con kernel lineal




In [7]:
def emoformat(A, emo):
    s = "  ".join([a[0] for a in A if a[1] == emo])
    return s[:1000] + "..."
    
with open('microTC/microtc/resources/emoticons.json') as f:
    A = list(json.load(f).items())
    A.sort()
    S = dict(
        pos=emoformat(A, '_pos'),
        neg=emoformat(A, '_neg'),
        neu=emoformat(A, '_neu'),
        none=emoformat(A, '_none'),
    )

output = ["## Emoticones y emojis clasificados por sentimiento ##"]
for k, v in S.items():
    output.append("## Clase `{0}` ##".format(k.upper()))
    output.append(v)

Markdown("\n".join(output))

## Emoticones y emojis clasificados por sentimiento ##
## Clase `NONE` ##
'~'U  ^( \\\'-\\\' )^  π  ϟ  ಠ~ಠ  ‷̗ↂ凸ↂ‴  ‼  ⁂  ⁉  ⁎  ⁑  ℹ  ↶  ↷  ∨  ∩  ∪  ∴  ⊕  ⊗  ⊚  ⊛  ⊜  ⊥  ⌆  ⌘  ⌥  ⌫  ⎈  ⎋  ⎌  ⏎  ⏏  ╽  ╿  ◉  ●  ◐  ◑  ◒  ◓  ◔  ◕  ◖  ◗  ☄  ☇  ☈  ☉  ☊  ☌  ☍  ☎  ☏  ☛  ☝  ☞  ☤  ☧  ☨  ☩  ☪  ☫  ☬  ☭  ☸  ☿  ♊  ♋  ♌  ♍  ♎  ♏  ♐  ♑  ♒  ♓  ♤  ♨  ♭  ♮  ♯  ♿  ⚒  ⚓  ⚔  ⚕  ⚘  ⚚  ⚜  ⚪  ⚫  ✆  ✇  ✊  ✑  ✠  ✡  ✢  ✣  ✤  ✥  ✦  ✧  ❖  ❗  ❦  ❧  ➖  ➗  ➰  ➷  ➺  ➿  づ  ゞ  ガ  ギ  グ  ゲ  ゴ  ザ  ズ  ゼ  バ  ビ  ヷ  ヹ  ヺ  ㊊  ㊋  ㊌  ㊍  ㊎  ㊏  ㊐  유  ﹢  ﹣  ﹤  ﹥  🀄...
## Clase `POS` ##
(^L^)  (^‿^)  (°⌣°)  (•‿•)  (｡◕‿◕｡)  *-*  :)  :*  :-)  :-*  :-D  :3  :B  :D  :]  :p  ;)  ;-)  ;D  < (^^,) >  <3  <【☯】‿【☯】>  =)  =D  =^.^=  D:  Dx  XP  ^( \\\'‿\\\' )^  ^.^  ^^  ^_^  ^o^  ^‿^  n_n  q(❂‿❂)p  xD  {◕ ◡ ◕}  |◔◡◉|  ¢‿¢  Ü  ñ.ñ  ñ__ñ  ñ_ñ  ó‿ó  ت  ٩(-̮̮̃-̃)۶  ٩(-̮̮̃•̃)  ٩(^‿^)۶  ٩(͡๏̮͡๏)۶  ٩◔‿◔۶  ಠ◡ಠ  ༺‿༻  ღ  ᵔᴥᵔ  ‍👨❤‍️‍💋👨  ‍👩❤‍️‍💋👨  ‍👩❤‍️‍💋👩  †  ⇧  ⇪  √  ≧◡≦  ⊂◉‿◉つ  ⊙▃⊙  ⋆  ⍟  ┌(ಠ‿ಠ)┘  ╳  ●‿●  ◕‿◕٩(●̮̮̃•̃)۶  ◙‿◙  ☀  ☃  ★  ☆  ☋  ☑  ☕  ☘  ☚  ☜  ☥  ☦  ☮  ☯  ☺  ☻  ☼  ☽  ☾  ♉  ♔  ♕  ♖  ♚  ♛  ♡  ♣  ♥  ♥‿♥  ♧  ♩  ♪  ♫  ♬  ♻  ⚖  ⚛  ⛹  ⛹🏻  ⛹🏼  ⛹🏽  ⛹🏾  ⛹🏿  ✅  ✈  ✉  ✌  ✌🏻  ✌🏼  ✌🏽  ✌🏾  ✌🏿  ✓  ✔  ✙  ✚  ✛  ✜  ✝  ✞  ✟  ✨  ✩  ✪  ✫  ✬  ✭  ✮  ✯  ✰  ✱  ✲  ✳  ✴  ✵  ✶  ✷  ✸  ✹  ✺  ✻  ✼  ✽  ✾  ✿  ❀  ❀‿❀  ❁  ❂  ❃  ❇  ❈  ❉  ❊  ❋  ❕  ❣  ❤  ❤️  ❤️‿❤️  ❥  ➕  ➳  ➵  ➸  ➹  ➻  ➼  ➽  ⭐  シ  ジ  ゾ  ッ  ツ  ヅ  ㋡  乂◜◬◝乂  웃  🌄  🌅  🌷  🌹  🌺  🌻  🌼  🍀  🍻  🍾  🎂  🎄  🎆  🎇  🎈  🎉  🎊  🎑  🎖  🎟  🎧  🎫  🎵  🎶  🎼  🏅  🏆  🏩  🏵  🐭  🐮  🐱  🐵  👋  👋🏻  👋🏼  👋🏽  👋🏾  👋🏿  👌  👌🏻  👌🏼  👌🏽  👌🏾  👌🏿  👍  👍🏻  👍🏼  👍🏽  👍🏾  👍🏿  👎  👎🏻  👎🏼  👎🏽  👎🏾  👎🏿  👏...
## Clase `NEG` ##
'n'  (>.<)  (O.O)  (⊙̃.o ⊙.◎)  )-:  ):  ):-/  .l.  /:  :"(  :'(  :(  :-(  :-/  :-\  :-o  :-x  :/  :S  :[  :\  :c  :o  ='(  =(  =S  >.<  >:-(  >:o  ><  >_<  O.o  O.ó  TT  XC  X_X  \\\\m/(>.<)\\\\m/  o.Ó  õ.O  x_x  ¬ ¬  ¬ ¬*  ¬¬*  ».«  ò_ó  ॓_॔  ๏_๏  ⊝  ⌛  ⌤  ☂  ☒  ☔  ☟  ☠  ☢  ☣  ☹  ⚠  ⚡  ⛔  ✂  ✊🏻  ✊🏼  ✊🏽  ✊🏾  ✊🏿  ✋  ✕  ✖  ✗  ✘  ❌  ❎  〴⋋_⋌〵  卍  卐  🌁  🎗  👹  👺  👻  👽  👾  👿  💀  💔  💢  💣  💦  💨  💩  💫  📉  🖕  🖕🏻  🖕🏼  🖕🏽  🖕🏾  🖕🏿  🗯  😓  😔  😖  😞  😟  😠  😡  😢  😣  😥  😦  😧  😨  😩  😪  😫  😬  😭  😮  😯  😰  😱  😲  😳  😵  😷  😾  😿  🙀  🙁  🙃  🙅  🙅🏻  🙅🏼  🙅🏽  🙅🏾  🙅🏿  🙈  🙉  🙊  🙎  🙎🏻  🙎🏼  🙎🏽  🙎🏾  🙎🏿  🚫  🚳  🤒  🤕  🤘  🤘🏻  🤘🏼  🤘🏽  🤘🏾  🤘🏿...
## Clase `NEU` ##
#⃣  $_$  &  (-\\\'\\\'-)  (•̪●)  *⃣  -.-  -_-  ._.  0⃣  1⃣  2⃣  3⃣  4⃣  5⃣  6⃣  7⃣  8-|  8⃣  9⃣  :-|  >:-o  @_@  U_U  n__n  u.u  u_u  v( ‘.’ )v  {•̃̾_•̃̾}  |˚–˚|  ©  ¬¬  ®  ఠ_ఠ  ರ_ರ  ‍👨❤‍️👨  ‍👩❤‍️👨  ‍👩❤‍️👩  ™  ↔  ↕  ↖  ↗  ↘  ↙  ↩  ↪  ⇎_⇎  ⌚  ⌨  ⏩  ⏪  ⏫  ⏬  ⏭  ⏮  ⏯  ⏰  ⏱  ⏲  ⏳  ⏸  ⏹  ⏺  Ⓜ  ▪  ▫  ▶  ◀  ◐.̃◐  ◑.◑  ◔_◔  ◻  ◼  ◽  ◾  ☁  ☝🏻  ☝🏼  ☝🏽  ☝🏾  ☝🏿  ♀  ♁  ♂  ♈  ♗  ♘  ♙  ♜  ♝  ♞  ♟  ♠  ♢  ♦  ⚗  ⚙  ⚰  ⚱  ⚽  ⚾  ⛄  ⛅  ⛈  ⛎  ⛏  ⛑  ⛓  ⛩  ⛪  ⛰  ⛱  ⛲  ⛳  ⛴  ⛵  ⛷  ⛸  ⛺  ⛽  ✋🏻  ✋🏼  ✋🏽  ✋🏾  ✋🏿  ✍  ✍🏻  ✍🏼  ✍🏽  ✍🏾  ✍🏿  ✎  ✏  ✒  ❄  ❅  ❆  ❓  ❔  ➡  ⤴  ⤵  ⬅  ⬆  ⬇  ⬛  ⬜  ⭕  【•】_【•】  〰  〽  ㊗  ㊙  🃏  🅰  🅱  🅾  🅿  🆎  🆑  🆒  🆓  🆔  🆕  🆖  🆗  🆘  🆙  🆚  🇦🇨  🇦🇩  🇦🇪  🇦🇫  🇦🇬  🇦🇮  🇦🇱  🇦🇲  🇦🇴  🇦🇶  🇦🇷  🇦🇸  🇦🇹  🇦🇺  🇦🇼  🇦🇽  🇦🇿  🇧🇦  🇧🇧  🇧🇩  🇧🇪  🇧🇫  🇧🇬  🇧🇭  🇧🇮  🇧🇯  🇧🇱  🇧🇲  🇧🇳  🇧🇴  🇧🇶  🇧🇷  🇧🇸  🇧🇹  🇧🇻  🇧🇼  🇧🇾  🇧🇿  🇨🇦  🇨🇨  🇨🇩  🇨🇫  🇨🇬  🇨🇭  🇨🇮  🇨🇰  🇨🇱  🇨🇲  🇨🇳  🇨🇴  🇨🇵  🇨🇷  🇨🇺  🇨🇻  🇨🇼  🇨🇽  🇨🇾  🇨🇿  🇩🇪  🇩🇬  🇩🇯  🇩🇰  🇩🇲  🇩🇴  🇩🇿  🇪🇦  🇪🇨  🇪🇪  🇪🇬  🇪🇭  🇪🇷  🇪🇸  🇪🇹  🇪🇺  🇫🇮  🇫🇯  🇫🇰  🇫🇲  🇫🇴  🇫🇷  🇬🇦  🇬🇧  🇬🇩  🇬🇪  🇬🇫  🇬🇬  🇬🇭  🇬🇮  🇬🇱  🇬🇲  🇬🇳  🇬🇵 ...

## Tokenizadores Ejemplos ##

In [9]:
text = "anita lava la tina"
tokenizers = [1, 2, 3, -1, -2, (2,1), (2,2)]

num = 1
output = []
for ltokens in range(len(tokenizers)):
    output.append('## Combinaciones de tamaño {0} ##'.format(ltokens+1))
    output.append('|id|combinación|tokens|')
    output.append('|--|-----------|------|')
    for comb in combinations(tokenizers, ltokens+1):
        model = TextModel([], token_list=comb)
        output.append("|{0}|{1}|{2}|".format(num, comb, ", ".join(model.tokenize(text))))
        num += 1

Markdown("\n".join(output))

## Combinaciones de tamaño 1 ##
|id|combinación|tokens|
|--|-----------|------|
|1|(1,)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~|
|2|(2,)|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~|
|3|(3,)|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~|
|4|(-1,)|anita, lava, la, tina|
|5|(-2,)|anita~lava, lava~la, la~tina|
|6|((2, 1),)|anita~la, lava~tina|
|7|((2, 2),)|anita~tina|
## Combinaciones de tamaño 2 ##
|id|combinación|tokens|
|--|-----------|------|
|8|(1, 2)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~|
|9|(1, 3)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~|
|10|(1, -1)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita, lava, la, tina|
|11|(1, -2)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita~lava, lava~la, la~tina|
|12|(1, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita~la, lava~tina|
|13|(1, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita~tina|
|14|(2, 3)|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~|
|15|(2, -1)|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina|
|16|(2, -2)|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~lava, lava~la, la~tina|
|17|(2, (2, 1))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~la, lava~tina|
|18|(2, (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~tina|
|19|(3, -1)|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina|
|20|(3, -2)|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina|
|21|(3, (2, 1))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~la, lava~tina|
|22|(3, (2, 2))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~tina|
|23|(-1, -2)|anita, lava, la, tina, anita~lava, lava~la, la~tina|
|24|(-1, (2, 1))|anita, lava, la, tina, anita~la, lava~tina|
|25|(-1, (2, 2))|anita, lava, la, tina, anita~tina|
|26|(-2, (2, 1))|anita~lava, lava~la, la~tina, anita~la, lava~tina|
|27|(-2, (2, 2))|anita~lava, lava~la, la~tina, anita~tina|
|28|((2, 1), (2, 2))|anita~la, lava~tina, anita~tina|
## Combinaciones de tamaño 3 ##
|id|combinación|tokens|
|--|-----------|------|
|29|(1, 2, 3)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~|
|30|(1, 2, -1)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina|
|31|(1, 2, -2)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~lava, lava~la, la~tina|
|32|(1, 2, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~la, lava~tina|
|33|(1, 2, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~tina|
|34|(1, 3, -1)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina|
|35|(1, 3, -2)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina|
|36|(1, 3, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~la, lava~tina|
|37|(1, 3, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~tina|
|38|(1, -1, -2)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita, lava, la, tina, anita~lava, lava~la, la~tina|
|39|(1, -1, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita, lava, la, tina, anita~la, lava~tina|
|40|(1, -1, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita, lava, la, tina, anita~tina|
|41|(1, -2, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|42|(1, -2, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita~lava, lava~la, la~tina, anita~tina|
|43|(1, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita~la, lava~tina, anita~tina|
|44|(2, 3, -1)|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina|
|45|(2, 3, -2)|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina|
|46|(2, 3, (2, 1))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~la, lava~tina|
|47|(2, 3, (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~tina|
|48|(2, -1, -2)|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~lava, lava~la, la~tina|
|49|(2, -1, (2, 1))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~la, lava~tina|
|50|(2, -1, (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~tina|
|51|(2, -2, (2, 1))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|52|(2, -2, (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~lava, lava~la, la~tina, anita~tina|
|53|(2, (2, 1), (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~la, lava~tina, anita~tina|
|54|(3, -1, -2)|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina|
|55|(3, -1, (2, 1))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~la, lava~tina|
|56|(3, -1, (2, 2))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~tina|
|57|(3, -2, (2, 1))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|58|(3, -2, (2, 2))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~tina|
|59|(3, (2, 1), (2, 2))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~la, lava~tina, anita~tina|
|60|(-1, -2, (2, 1))|anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|61|(-1, -2, (2, 2))|anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~tina|
|62|(-1, (2, 1), (2, 2))|anita, lava, la, tina, anita~la, lava~tina, anita~tina|
|63|(-2, (2, 1), (2, 2))|anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
## Combinaciones de tamaño 4 ##
|id|combinación|tokens|
|--|-----------|------|
|64|(1, 2, 3, -1)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina|
|65|(1, 2, 3, -2)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina|
|66|(1, 2, 3, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~la, lava~tina|
|67|(1, 2, 3, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~tina|
|68|(1, 2, -1, -2)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~lava, lava~la, la~tina|
|69|(1, 2, -1, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~la, lava~tina|
|70|(1, 2, -1, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~tina|
|71|(1, 2, -2, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|72|(1, 2, -2, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~lava, lava~la, la~tina, anita~tina|
|73|(1, 2, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~la, lava~tina, anita~tina|
|74|(1, 3, -1, -2)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina|
|75|(1, 3, -1, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~la, lava~tina|
|76|(1, 3, -1, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~tina|
|77|(1, 3, -2, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|78|(1, 3, -2, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~tina|
|79|(1, 3, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~la, lava~tina, anita~tina|
|80|(1, -1, -2, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|81|(1, -1, -2, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~tina|
|82|(1, -1, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita, lava, la, tina, anita~la, lava~tina, anita~tina|
|83|(1, -2, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|84|(2, 3, -1, -2)|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina|
|85|(2, 3, -1, (2, 1))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~la, lava~tina|
|86|(2, 3, -1, (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~tina|
|87|(2, 3, -2, (2, 1))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|88|(2, 3, -2, (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~tina|
|89|(2, 3, (2, 1), (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~la, lava~tina, anita~tina|
|90|(2, -1, -2, (2, 1))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|91|(2, -1, -2, (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~tina|
|92|(2, -1, (2, 1), (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~la, lava~tina, anita~tina|
|93|(2, -2, (2, 1), (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|94|(3, -1, -2, (2, 1))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|95|(3, -1, -2, (2, 2))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~tina|
|96|(3, -1, (2, 1), (2, 2))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~la, lava~tina, anita~tina|
|97|(3, -2, (2, 1), (2, 2))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|98|(-1, -2, (2, 1), (2, 2))|anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
## Combinaciones de tamaño 5 ##
|id|combinación|tokens|
|--|-----------|------|
|99|(1, 2, 3, -1, -2)|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina|
|100|(1, 2, 3, -1, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~la, lava~tina|
|101|(1, 2, 3, -1, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~tina|
|102|(1, 2, 3, -2, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|103|(1, 2, 3, -2, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~tina|
|104|(1, 2, 3, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~la, lava~tina, anita~tina|
|105|(1, 2, -1, -2, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|106|(1, 2, -1, -2, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~tina|
|107|(1, 2, -1, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~la, lava~tina, anita~tina|
|108|(1, 2, -2, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|109|(1, 3, -1, -2, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|110|(1, 3, -1, -2, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~tina|
|111|(1, 3, -1, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~la, lava~tina, anita~tina|
|112|(1, 3, -2, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|113|(1, -1, -2, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|114|(2, 3, -1, -2, (2, 1))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|115|(2, 3, -1, -2, (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~tina|
|116|(2, 3, -1, (2, 1), (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~la, lava~tina, anita~tina|
|117|(2, 3, -2, (2, 1), (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|118|(2, -1, -2, (2, 1), (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|119|(3, -1, -2, (2, 1), (2, 2))|~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
## Combinaciones de tamaño 6 ##
|id|combinación|tokens|
|--|-----------|------|
|120|(1, 2, 3, -1, -2, (2, 1))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina|
|121|(1, 2, 3, -1, -2, (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~tina|
|122|(1, 2, 3, -1, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~la, lava~tina, anita~tina|
|123|(1, 2, 3, -2, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|124|(1, 2, -1, -2, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|125|(1, 3, -1, -2, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
|126|(2, 3, -1, -2, (2, 1), (2, 2))|~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|
## Combinaciones de tamaño 7 ##
|id|combinación|tokens|
|--|-----------|------|
|127|(1, 2, 3, -1, -2, (2, 1), (2, 2))|~, a, n, i, t, a, ~, l, a, v, a, ~, l, a, ~, t, i, n, a, ~, ~a, an, ni, it, ta, a~, ~l, la, av, va, a~, ~l, la, a~, ~t, ti, in, na, a~, ~an, ani, nit, ita, ta~, a~l, ~la, lav, ava, va~, a~l, ~la, la~, a~t, ~ti, tin, ina, na~, anita, lava, la, tina, anita~lava, lava~la, la~tina, anita~la, lava~tina, anita~tina|